In [19]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
!pip install SpeechRecognition
import speech_recognition as sr
import soundfile
!pip install ffmpeg

r = sr.Recognizer()
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=10):
  print('starting')
  display(Javascript(RECORD))
  
  s = output.eval_js('record(%d)' % (sec*1000))
  print('end')
  b = b64decode(s.split(',')[1])
  with open('audio.webm','wb') as f:
    f.write(b)
  
  return 'audio.webm'  # or webm ?

In [20]:
record()

starting


<IPython.core.display.Javascript object>

end


'audio.webm'

In [21]:
#Processing from webm to the correct wav format
!ffmpeg -i ./audio.webm -c:a pcm_f32le -y ./out.wav
data, samplerate = soundfile.read('out.wav')
soundfile.write('audio2.wav', data, samplerate, subtype='PCM_16')

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [25]:
with sr.AudioFile('audio2.wav') as source:
  audio_text = r.listen(source)
  text = r.recognize_google(audio_text)
  # print('Converting audio transcripts into text ...')
  print(text)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
